In [ ]:
import json 











In [ ]:
import json

path = "/data/userdata/v-lijingyuan/dpo/final_data_diff_1.json"

with open(path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(data[:10])  # 如果是 list


In [ ]:
def load_jsonl_as_list(path):
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for idx, line in enumerate(f):
            line = line.strip()
            if not line:
                continue
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"[SKIP] line {idx}: {e}")
    return data

In [ ]:
import json

src_path = "/data/userdata/v-lijingyuan/dpo/final_data_diff_1.json"
mapping_path = "/data/userdata/v-lijingyuan/rl_pipe_line/final_teacher_chain_v4.jsonl"
#out_path = "/data/userdata/v-lijingyuan/dpo/final_data_diff_2.json"

mapping_data =  load_jsonl_as_list(mapping_path)


mapping = {}
for obj in mapping_data:
    key = (obj["comptation_name"], obj["hypothesis_chain"])
    mapping[key] = obj["new_hypothesis_chain"]

print(f"Loaded {len(mapping)} hypothesis mappings")

# ===== 2. 读取原始 DPO 数据 =====
with open(src_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"Loaded {len(data)} DPO samples")

# ===== 3. 批量替换 =====
replaced_winner = 0
replaced_loser = 0

for idx, sample in enumerate(data):
    comp = sample.get("comptation_name")

    # 打标识（可选）
    sample["_replace_flag"] = {"winner": False, "loser": False}

    # winner
    if "winner" in sample:
        key = (comp, sample["winner"])
        if key in mapping:
            print(f"[MATCH][{idx}] WINNER | {comp}")
            sample["winner"] = mapping[key]
            sample["_replace_flag"]["winner"] = True
            replaced_winner += 1

    # loser
    if "loser" in sample:
        key = (comp, sample["loser"])
        if key in mapping:
            print(f"[MATCH][{idx}] LOSER  | {comp}")
            sample["loser"] = mapping[key]
            sample["_replace_flag"]["loser"] = True
            replaced_loser += 1

# # ===== 4. 写回 =====
# with open(out_path, "w", encoding="utf-8") as f:
#     json.dump(data, f, ensure_ascii=False, indent=2)

print(f"Replaced winner: {replaced_winner}")
print(f"Replaced loser : {replaced_loser}")
#print(f"Saved to {out_path}")


In [ ]:
data[0]

In [ ]:
import json
from typing import List, Dict


# =========================
# utils
# =========================
def load_jsonl_as_list(path: str) -> List[Dict]:
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"[WARN] skip bad line {i}: {e}")
    return data


# =========================
# paths
# =========================
src_path = "/data/userdata/v-lijingyuan/dpo/final_data_diff_1.json"
mapping_path = "/data/userdata/v-lijingyuan/rl_pipe_line/final_teacher_chain_v4.jsonl"
out_path = "/data/userdata/v-lijingyuan/dpo/final_data_diff_2.json"


# =========================
# 1. load mapping (jsonl)
# =========================
mapping_data = load_jsonl_as_list(mapping_path)

mapping = {}
for obj in mapping_data:
    key = (obj["comptation_name"], obj["hypothesis_chain"])
    mapping[key] = obj["new_hypothesis_chain"]

print(f"[INFO] Loaded {len(mapping)} hypothesis mappings")


# =========================
# 2. load original DPO data
# =========================
with open(src_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"[INFO] Loaded {len(data)} DPO samples")


# =========================
# 3. batch replace + collect examples
# =========================
replaced_winner = 0
replaced_loser = 0

examples = []
MAX_EXAMPLES = 1000

for idx, sample in enumerate(data):
    comp = sample.get("comptation_name")

    # mark replace flag
    sample["_replace_flag"] = {"winner": False, "loser": False}

    # -------- winner --------
    if "winner" in sample:
        key = (comp, sample["winner"])
        if key in mapping:
            old = sample["winner"]
            new = mapping[key]

            sample["winner"] = new
            sample["_replace_flag"]["winner"] = True
            replaced_winner += 1

            if len(examples) < MAX_EXAMPLES:
                examples.append({
                    "idx": idx,
                    "type": "winner",
                    "competition": comp,
                    "old": old,
                    "new": new,
                })

    # -------- loser --------
    if "loser" in sample:
        key = (comp, sample["loser"])
        if key in mapping:
            old = sample["loser"]
            new = mapping[key]

            sample["loser"] = new
            sample["_replace_flag"]["loser"] = True
            replaced_loser += 1

            if len(examples) < MAX_EXAMPLES:
                examples.append({
                    "idx": idx,
                    "type": "loser",
                    "competition": comp,
                    "old": old,
                    "new": new,
                })


# =========================
# 4. print replacement examples
# =========================
print("\n========== REPLACEMENT EXAMPLES ==========")
for i, ex in enumerate(examples, 1):
    print(f"\n[EXAMPLE {i}] [{ex['type'].upper()}]")
    print(f"idx        : {ex['idx']}")
    print(f"competition: {ex['competition']}")
    print("OLD:")
    print(ex["old"])
    print("NEW:")
    print(ex["new"])


# =========================
# 5. stats
# =========================
print("\n========== STATS ==========")
print(f"Replaced winner: {replaced_winner}")
print(f"Replaced loser : {replaced_loser}")
print(f"Total samples  : {len(data)}")


# =========================
# 6. save result
# ==================


In [ ]:
examples

In [ ]:



scp -r /data/userdata/v-lijingyuan/train_reward_stage3_3/last_run_7 \
      v-lijingyuan@ep12.213428.xyz:/home/v-lijingyuan/train_aira

In [ ]:
import matplotlib.pyplot as plt



# =========================
# OLD TEXT（你给的内容，已示例性标红）
# =========================
old_text = [

]


# =========================
# NEW TEXT（占位，你自己填）
# =========================
new_text = (
    "Load a class-stratified random subset of the training data "
    "(using a fixed random seed) for rapid experimentation and "
    "memory-efficient local validation. "
    "-> Introduce lightweight data-augmentation or synthetic resampling "
    "strategies to mitigate class imbalance and enrich the learning "
    "signal during early prototyping. "
    "-> After validating modeling choices, fit a robust ensemble model "
    "on the complete dataset, leveraging cross-validated early stopping "
    "and class-balanced metrics to prevent overfitting and ensure fair "
    "performance across all classes."
)
# 自动换行
old_text = fill(old_text, width=100)
new_text = fill(new_text, width=100)

# =========================
# 绘图
# =========================
plt.rcParams["text.usetex"] = True
plt.rcParams["font.size"] = 10

fig, ax = plt.subplots(figsize=(11, 6))
ax.axis("off")

ax.text(
    0.01, 0.95,
    r"\textbf{Old Text (Overly Localized Feedback)}" + "\n\n" + old_text,
    va="top",
    ha="left",
    bbox=dict(boxstyle="round,pad=0.5", fc="#f2f2f2", ec="black")
)

ax.text(
    0.01, 0.40,
    r"\textbf{New Text (Improved Reasoning)}" + "\n\n" + new_text,
    va="top",
    ha="left",
    bbox=dict(boxstyle="round,pad=0.5", fc="white", ec="black")
)

plt.tight_layout()
plt.show()
#plt.savefig("localized_feedback_old_new.pdf", bbox_inches="tight")
#plt.savefig("localized_feedback_old_new.png", dpi=300, bbox_inches="tight")
#plt.close()

print("Saved: localized_feedback_old_new.pdf / localized_feedback_old_new.png")


In [ ]:
import matplotlib.pyplot as plt
from textwrap import fill

# ===== 你自己填的内容 =====
old_text = (
"Load a class-stratified random subset of 300,000 rows (random_state=42) from train.csv for training/validation to cut runtime/memory while preserving class distribution for trustworthy quick tests.->Add lightweight domain features and sanitize out-of-range values, then retrain the same LightGBM on the 300k stratified subset. Specifically: (1) Create sin_aspect = sin((Aspect % 360) * pi/180) and cos_aspect = cos((Aspect % 360) * pi/180) to capture circular geometry; (2) Create elev_minus_vdh = Elevation - Vertical_Distance_To_Hydrology; (3) Create abs_vdh = abs(Vertical_Distance_To_Hydrology); (4) Create hydrology_dist = sqrt(Horizontal_Distance_To_Hydrology^2 + Vertical_Distance_To_Hydrology^2); (5) Create road_fire_diff = Horizontal_Distance_To_Roadways - Horizontal_Distance_To_Fire_Points and road_fire_sum = Horizontal_Distance_To_Roadways + Horizontal_Distance_To_Fire_Points. Sanitize values consistently in train/test: clip all Horizontal_Distance_* features at >= 0, keep Vertical_Distance_To_Hydrology signed (do not clip), and clip Hillshade_9am/Noon/3pm into [0, 255]. Keep all other training settings unchanged.->Leverage the full dataset with GPU-accelerated boosting and align early stopping with Accuracy. Specifically, replace the current CPU LightGBM trained on a 300k subsample with XGBoost GPU trained on all 3.6M rows while keeping the existing sanitation and engineered features. Implement a minimal-count–aware 90/10 holdout: route all samples of any class with count <= 5 entirely to training, then perform a stratified 90/10 split on the remaining rows. Map labels 1–7 to 0–6 for training. Train XGBoost with tree_method=gpu_hist, predictor=gpu_predictor, objective=multi:softprob, num_class=7, eval_metric=merror, early_stopping_rounds=100, and sensible GPU-friendly params (e.g., max_depth=8, learning_rate=0.05, subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0, n_estimators up to 3000, single_precision_histogram=1). Use the best_iteration to predict class labels on the full test set and map back to 1–7. This reduces sampling noise, exploits the available GPU, and optimizes directly for Accuracy while safeguarding rare classes in the split."
)

new_text = (
    "Load a class-stratified random subset of the training data "
    "(using a fixed random seed) for rapid experimentation and "
    "memory-efficient local validation. "
    "-> Introduce lightweight data-augmentation or synthetic resampling "
    "strategies to mitigate class imbalance and enrich the learning "
    "signal during early prototyping. "
    "-> After validating modeling choices, fit a robust ensemble model "
    "on the complete dataset, leveraging cross-validated early stopping "
    "and class-balanced metrics to prevent overfitting and ensure fair "
    "performance across all classes."
)
# ===========================

# 自动换行，避免超出边界
old_text = fill(old_text, width=100)
new_text = fill(new_text, width=100)

fig, ax = plt.subplots(figsize=(14, 6))
ax.axis("off")

# Old text box
ax.text(
    0.02, 0.95,
    "Hypothesis \n\n" + old_text,
    fontsize=9,
    va="top",
    ha="left",
    linespacing=1.4,
    bbox=dict(
        boxstyle="round,pad=0.6",
        fc="#eeeeee",      # 浅灰，更柔和
        ec="#444444",      # 深灰边框
        lw=1.0
    )
)

# ===== New text box =====
ax.text(
    0.75, 0.95,
    "New Hypothesis chian \n\n" + new_text,
    fontsize=9,
    va="top",
    ha="left",
    linespacing=1.4,
    bbox=dict(
        boxstyle="round,pad=0.6",
        fc="#ffffff",      # 白底，突出“改进”
        ec="#444444",
        lw=1.0
    )
)
plt.tight_layout()
plt.show()
#plt.savefig("localized_feedback_example.png", dpi=300, bbox_inches="tight")
#plt.savefig("localized_feedback_example.pdf", bbox_inches="tight")
#plt.close()

#print("Saved: localized_feedback_example.png / .pdf")


In [ ]:
old_text_lines = [

    [
        ("Load a class-stratified random subset of ", "black"),
        ("300,000", "red"),
        (" rows (", "black"),
        ("random_state=42", "red"),
        (") from train.csv for training/validation to cut runtime/memory "
         "while preserving class distribution for trustworthy quick tests.", "black"),
    ],

    [
        ("→ Add lightweight domain features and sanitize out-of-range values, "
         "then retrain the same LightGBM on the ", "black"),
        ("300k", "red"),
        (" stratified subset. Specifically:", "black"),
    ],

    [
        ("(1) Create ", "black"),
        ("sin_aspect = sin((Aspect % 360) * pi/180)", "red"),
        (" and ", "black"),
        ("cos_aspect = cos((Aspect % 360) * pi/180)", "red"),
        (" to capture circular geometry;", "black"),
    ],

    [
        ("(2) Create ", "black"),
        ("elev_minus_vdh = Elevation - Vertical_Distance_To_Hydrology", "red"),
        (";", "black"),
    ],

    [
        ("(3) Create ", "black"),
        ("abs_vdh = abs(Vertical_Distance_To_Hydrology)", "red"),
        (";", "black"),
    ],

    [
        ("(4) Create ", "black"),
        ("hydrology_dist = sqrt(Horizontal_Distance_To_Hydrology^2 + "
         "Vertical_Distance_To_Hydrology^2)", "red"),
        (";", "black"),
    ],

    [
        ("(5) Create ", "black"),
        ("road_fire_diff = Horizontal_Distance_To_Roadways - "
         "Horizontal_Distance_To_Fire_Points", "red"),
        (" and ", "black"),
        ("road_fire_sum = Horizontal_Distance_To_Roadways + "
         "Horizontal_Distance_To_Fire_Points", "red"),
        (".", "black"),
    ],

    [
        ("Sanitize values consistently in train/test: clip all ", "black"),
        ("Horizontal_Distance_* >= 0", "red"),
        (", keep ", "black"),
        ("Vertical_Distance_To_Hydrology signed", "red"),
        (" (do not clip), and clip ", "black"),
        ("Hillshade_9am/Noon/3pm into [0, 255]", "red"),
        (". Keep all other training settings unchanged.", "black"),
    ],

    [
        ("→ Leverage the full dataset with ", "black"),
        ("GPU-accelerated boosting", "red"),
        (" and align early stopping with ", "black"),
        ("Accuracy", "red"),
        (". Specifically, replace the current CPU LightGBM trained on a ", "black"),
        ("300k", "red"),
        (" subsample with ", "black"),
        ("XGBoost GPU", "red"),
        (" trained on all ", "black"),
        ("3.6M", "red"),
        (" rows.", "black"),
    ],

    [
        ("Train XGBoost with ", "black"),
        ("tree_method=gpu_hist", "red"),
        (", ", "black"),
        ("predictor=gpu_predictor", "red"),
        (", ", "black"),
        ("objective=multi:softprob", "red"),
        (", ", "black"),
        ("num_class=7", "red"),
        (", ", "black"),
        ("early_stopping_rounds=100", "red"),
        (", ", "black"),
        ("max_depth=8", "red"),
        (", ", "black"),
        ("learning_rate=0.05", "red"),
        (", ", "black"),
        ("subsample=0.8", "red"),
        (", ", "black"),
        ("colsample_bytree=0.8", "red"),
        (", ", "black"),
        ("n_estimators=3000", "red"),
        (".", "black"),
    ],

    [
        ("Use the best_iteration to predict class labels on the full test set "
         "and map back to 1–7. This reduces sampling noise, exploits the available GPU, "
         "and optimizes directly for Accuracy while safeguarding rare classes in the split.", "black"),
    ],
]


import matplotlib.pyplot as plt
from matplotlib.offsetbox import TextArea, HPacker, VPacker, AnchoredOffsetbox

fig, ax = plt.subplots(figsize=(10, 5))
ax.axis("off")

line_boxes = []

for line in old_text_lines:
    areas = [
        TextArea(
            txt,
            textprops=dict(color=color, fontsize=9)
        )
        for txt, color in line
    ]
    line_boxes.append(
        HPacker(children=areas, align="baseline", pad=0, sep=2)
    )

vbox = VPacker(children=line_boxes, align="left", pad=4, sep=6)

anchored_box = AnchoredOffsetbox(
    loc="upper left",
    child=vbox,
    pad=0.4,
    frameon=True,
    bbox_to_anchor=(0.02, 0.95),
    bbox_transform=ax.transAxes,
)

anchored_box.patch.set_boxstyle("round,pad=0.5")
anchored_box.patch.set_facecolor("#eeeeee")
anchored_box.patch.set_edgecolor("#2B2424")

ax.add_artist(anchored_box)


plt.show()



In [ ]:
import matplotlib.pyplot as plt
from matplotlib.offsetbox import TextArea, HPacker, VPacker, AnchoredOffsetbox, DrawingArea
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle

fig, ax = plt.subplots(figsize=(11, 7))
ax.axis("off")

# =========================
# 1. Old text（富文本，部分红色）
# =========================

old_text_lines = [

    [
        ("Load a class-stratified random subset of ", "black"),
        ("300,000", "red"),
        (" rows (", "black"),
        ("random_state=42", "red"),
        (") from train.csv for training/validation to cut runtime/memory "
         "while preserving class distribution for trustworthy quick tests.", "black"),
    ],

    [
        ("→ Add lightweight domain features and sanitize out-of-range values, "
         "then retrain the same LightGBM on the ", "black"),
        ("300k", "red"),
        (" stratified subset. Specifically:", "black"),
    ],

    [
        ("(1) Create ", "black"),
        ("sin_aspect = sin((Aspect % 360) * pi/180)", "red"),
        (" and ", "black"),
        ("cos_aspect = cos((Aspect % 360) * pi/180)", "red"),
        (" to capture circular geometry;", "black"),
    ],

    [
        ("(2) Create ", "black"),
        ("elev_minus_vdh = Elevation - Vertical_Distance_To_Hydrology", "red"),
        (";", "black"),
    ],

    [
        ("(3) Create ", "black"),
        ("abs_vdh = abs(Vertical_Distance_To_Hydrology)", "red"),
        (";", "black"),
    ],

    [
        ("(4) Create ", "black"),
        ("hydrology_dist = sqrt(Horizontal_Distance_To_Hydrology^2 + "
         "Vertical_Distance_To_Hydrology^2)", "red"),
        (";", "black"),
    ],

    [
        ("(5) Create ", "black"),
        ("road_fire_diff = Horizontal_Distance_To_Roadways - "
         "Horizontal_Distance_To_Fire_Points", "red"),
        (" and ", "black"),
        ("road_fire_sum = Horizontal_Distance_To_Roadways + "
         "Horizontal_Distance_To_Fire_Points", "red"),
        (".", "black"),
    ],

    [
        ("Sanitize values consistently in train/test: clip all ", "black"),
        ("Horizontal_Distance_* >= 0", "red"),
        (", keep ", "black"),
        ("Vertical_Distance_To_Hydrology signed", "red"),
        (" (do not clip), and clip ", "black"),
        ("Hillshade_9am/Noon/3pm into [0, 255]", "red"),
        (". Keep all other training settings unchanged.", "black"),
    ],

    [
        ("→ Leverage the full dataset with ", "black"),
        ("GPU-accelerated boosting", "red"),
        (" and align early stopping with ", "black"),
        ("Accuracy", "red"),
        (". Specifically, replace the current CPU LightGBM trained on a ", "black"),
        ("300k", "red"),
        (" subsample with ", "black"),
        ("XGBoost GPU", "red"),
        (" trained on all ", "black"),
        ("3.6M", "red"),
        (" rows.", "black"),
    ],

    [
        ("Train XGBoost with ", "black"),
        ("tree_method=gpu_hist", "red"),
        (", ", "black"),
        ("predictor=gpu_predictor", "red"),
        (", ", "black"),
        ("objective=multi:softprob", "red"),
        (", ", "black"),
        ("num_class=7", "red"),
        (", ", "black"),
        ("early_stopping_rounds=100", "red"),
        (", ", "black"),
        ("max_depth=8", "red"),
        (", ", "black"),
        ("learning_rate=0.05", "red"),
        (", ", "black"),
        ("subsample=0.8", "red"),
        (", ", "black"),
        ("colsample_bytree=0.8", "red"),
        (", ", "black"),
        ("n_estimators=3000", "red"),
        (".", "black"),
    ],

    [
        ("Use the best_iteration to predict class labels on the full test set "
         "and map back to 1–7. This reduces sampling noise, exploits the available GPU, "
         "and optimizes directly for Accuracy while safeguarding rare classes in the split.", "black"),
    ],
]

old_line_boxes = []
for line in old_text_lines:
    areas = [
        TextArea(txt, textprops=dict(color=color, fontsize=9))
        for txt, color in line
    ]
    old_line_boxes.append(
        HPacker(children=areas, align="baseline", pad=0, sep=2)
    )

old_block = VPacker(
    children=old_line_boxes,
    align="left",
    pad=0,
    sep=5,
)

old_title = TextArea(
    "Original hypothesis chain\n",
    textprops=dict(fontsize=10, weight="bold")
)

# =========================
# 2. New text（普通文本）
# =========================

new_text = (
    "Load a class-stratified random subset of the training data "
    "(using a fixed random seed) for rapid experimentation and "
    "memory-efficient local validation.\n\n"
    "→ Introduce lightweight data-augmentation or synthetic resampling "
    "strategies to mitigate class imbalance and enrich the learning "
    "signal during early prototyping.\n\n"
    "→ After validating modeling choices, fit a robust ensemble model "
    "on the complete dataset, leveraging cross-validated early stopping "
    "and class-balanced metrics to prevent overfitting and ensure fair "
    "performance across all classes."
)

new_title = TextArea(
    "\nFuzzified hypothesis chain\n",
    textprops=dict(fontsize=10, weight="bold")
)

new_block = TextArea(
    new_text,
    textprops=dict(fontsize=9)
)

divider = DrawingArea(1, 1, 0, 0)
rect = Rectangle((0, 0.45), width=1100, height=0.7, color="black", transform=divider.get_transform())
divider.add_artist(rect)


vbox = VPacker(
    children=[
        old_title,
        old_block,
        divider,       # 分界线放在 Old / New 中间
        new_title,
        new_block,
    ],
    align="left",
    pad=6,
    sep=10,
)

anchored_box = AnchoredOffsetbox(
    loc="upper left",
    child=vbox,
    pad=0.4,
    frameon=True,
    bbox_to_anchor=(0.02, 0.95),
    bbox_transform=ax.transAxes,
)

anchored_box.patch.set_boxstyle("round,pad=0.6")
anchored_box.patch.set_facecolor("#f0f0f0")
anchored_box.patch.set_edgecolor("#2B2424")

ax.add_artist(anchored_box)
plt.savefig(
    "hypothesis_chain_comparison.png",  # 文件名
    dpi=300,                            # 高清
    bbox_inches='tight',                # 自动裁剪空白
    pad_inches=0.1,                     # 周围留一点边距
)
plt.show()
